In [12]:
import tensorflow as tf
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# from sklearn.model_selection import train_test_split
# import unicodedata
import re
import numpy as np
import os
import io
import time

In [13]:
import pandas as pd
lines = pd.read_csv('seq2seq_dataframe.csv')
lines.head()

,Unnamed: 0,Query,Prediction
0,0,mobicontrol manual mobicontrol manual,<START> サブタイトル 設定 mobicontrol 設定 <END>
1,1,サブタイトル 設定 mobicontrol 設定,<START> mobicontrol 設定 順序 端末 <END>
2,2,mobicontrol 設定 順序 端末,<START> サーバ 必要 設定 事項 <END>
3,3,サーバ 必要 設定 事項,<START> 必須 接続 プロファイル 作成 <END>
4,4,必須 接続 プロファイル 作成,<START> service サーバ 間 ad <END>


In [14]:
lines.head()
lines = lines.head(10)

In [ ]:
lines.tail()

In [16]:
num_examples = 30000
#input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(lines)
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Only for debugging and Visualization
# print ("Input Language; index to word mapping")
# convert(inp_lang, input_tensor[0])
# print (input_tensor[0], "=>", target_tensor[0])
# print ("Target Language; index to word mapping")
# convert(targ_lang, target_tensor[0])

# Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# BUFFER_SIZE = len(input_tensor_train)
BUFFER_SIZE = len(input_tensor)
BATCH_SIZE = 4
steps_per_epoch = len(input_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

# dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Only for debugging and Visualization
# print(vocab_inp_size, vocab_tar_size, steps_per_epoch)
# print(input_tensor_train)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state: (batch size, units) {}'.format(sample_hidden.shape))

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result: (batch size, units) {}".format(attention_result.shape))
print("Attention weights: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)
print ('Decoder output: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

EPOCHS = 25

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Encoder output: (batch size, sequence length, units) (4, 4, 1024)
Encoder Hidden state: (batch size, units) (4, 1024)
Attention result: (batch size, units) (4, 1024)
Attention weights: (batch_size, sequence_length, 1) (4, 4, 1)
Decoder output: (batch_size, vocab size) (4, 32)
Epoch 1 Batch 0 Loss 2.8883
Epoch 1 Loss 2.8866
Time taken for 1 epoch 5.18009090423584 sec

Epoch 2 Batch 0 Loss 2.8502
Epoch 2 Loss 2.8100
Time taken for 1 epoch 1.6031990051269531 sec

Epoch 3 Batch 0 Loss 2.6874
Epoch 3 Loss 2.6896
Time taken for 1 epoch 0.2818319797515869 sec

Epoch 4 Batch 0 Loss 2.5874
Epoch 4 Loss 2.5534
Time taken for 1 epoch 1.5217063426971436 sec

Epoch 5 Batch 0 Loss 2.6131
Epoch 5 Loss 2.4665
Time taken for 1 epoch 0.2781698703765869 sec

Epoch 6 Batch 0 Loss 2.3841
Epoch 6 Loss 2.4078
Time taken for 1 epoch 1.362687110900879 sec

Epoch 7 Batch 0 Loss 2.3895
Epoch 7 Loss 2.2448
Time taken for 1 epoch 0.2749958038330078 sec

Epoch 8 Batch 0 Loss 2.3624
Epoch 8 Loss 2.1578
Time taken fo

In [17]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [18]:
predict_sequence('mobicontrol manual android ios')

KeyError: 'android'

In [15]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state, attention_weights

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

def load_dataset(lines):
    targ_lang, inp_lang = lines.Prediction, lines.Query
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

def evaluate(sentence):
    # attention_plot = np.zeros((max_length_targ, max_length_inp))
    # sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)
        # storing the attention weights to plot later on
        # attention_weights = tf.reshape(attention_weights, (-1, ))
        # attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '
        # print(result)
        if targ_lang.index_word[predicted_id] == '<end>':
            # return result, sentence, attention_plot
            return result, sentence
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    # return result, sentence, attention_plot
    return result, sentence

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

def predict_sequence(sentence):
    # result, sentence, attention_plot = evaluate(sentence)
    result, sentence = evaluate(sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print("%d ----> %s" % (t, lang.index_word[t]))
